# CHATBOT PARA SERVICIO AL CLIENTE

A continuación se muestra el código para un chatbot para servicio al cliente. Su objetivo es tomar los terminos y condiciones de una aerolinea y en base a ellos contestar preguntas especificas referentes a esta información.

In [ ]:
!pip install sentence-transformers transformers python-docx scikit-learn numpy torch

In [ ]:
from sentence_transformers import CrossEncoder
re_ranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
import os
import time
import numpy as np
import torch
from docx import Document
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics.pairwise import cosine_similarity
import re

In [ ]:
def leer_docx_estructurado(path):
    doc = Document(path)
    parrafos = []

    for p in doc.paragraphs:
        estilo = p.style.name if p.style else ""
        texto = p.text.strip()
        if not texto:
            continue
        parrafos.append({"texto": texto, "estilo": estilo})
    return parrafos


def _es_lista(texto):
    return bool(re.match(r"^\s*[-•*]|\d+\.", texto.strip()))

In [ ]:
def extraer_chunks_por_titulo(parrafos):
    chunks = []
    actual_titulo = ""
    contenido_actual = []

    for p in parrafos:
        texto = p["texto"]
        estilo = p["estilo"]
        es_titulo = estilo.startswith("Heading") or re.match(r"^\d+(\.\d+)*\s", texto)

        if es_titulo:
            if actual_titulo and contenido_actual:
                chunk = f"{actual_titulo}\n" + "\n".join(contenido_actual)
                if len(chunk) > 40:
                    chunks.append(chunk)
            actual_titulo = texto
            contenido_actual = []
        else:
            contenido_actual.append(texto)

    # last chunk
    if actual_titulo and contenido_actual:
        chunk = f"{actual_titulo}\n" + "\n".join(contenido_actual)
        if len(chunk) > 40:
            chunks.append(chunk)

    return chunks

In [ ]:
def guardar_chunks_en_txt(chunks, ruta_salida="chunks_generados.txt"):
    with open(ruta_salida, "w", encoding="utf-8") as f:
        for i, chunk in enumerate(chunks):
            f.write(f"--- Chunk {i + 1} ---\n{chunk}\n\n")
    print(f"Chunks guardados")

In [ ]:
def buscar_chunks_relevantes(pregunta, chunks, embeddings_chunks, top_k=5, top_n_final=2):
    embedding_pregunta = modelo_embeddings.encode([pregunta], convert_to_tensor=True)
    pregunta_np = embedding_pregunta.cpu().numpy()
    chunks_np = embeddings_chunks.cpu().numpy()
    similitudes = cosine_similarity(pregunta_np, chunks_np)[0]

    indices_top_k = np.argsort(similitudes)[-top_k:]
    candidatos = [chunks[i] for i in indices_top_k]
    pares = [(pregunta, chunk) for chunk in candidatos]
    scores = re_ranker.predict(pares)

    indices_ordenados = np.argsort(scores)[::-1]  # Mayor a menor
    chunks_seleccionados = [candidatos[i] for i in indices_ordenados[:top_n_final]]

    return chunks_seleccionados, scores[indices_ordenados[0]]



def generar_respuesta(prompt, max_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=max_tokens,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.1,
        top_k=1,
        top_p=0.8
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
def responder_con_busqueda_semantica(pregunta, chunks, embeddings_chunks):
    contextos, score = buscar_chunks_relevantes(pregunta, chunks, embeddings_chunks, top_k=5, top_n_final=2)
    contexto = "\n\n".join(contextos)

    prompt = f"""You are a helpful assistant. Based ONLY on the documentation below, answer the question completely.

If there are multiple situations or scenarios in the documentation, then list and explain each scenario separately.

Use bullet points if needed and DO NOT add information that is not present in the documentation.

Documentation:
\"\"\"{contexto}\"\"\"

Question: {pregunta}
Answer:"""

    start = time.time()
    output = generar_respuesta(prompt)
    end = time.time()
    return output.split("Answer:")[-1].strip()

In [ ]:
modelo_embeddings = SentenceTransformer("all-MiniLM-L6-v2")

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")
model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b-instruct", torch_dtype=torch.float16, device_map="auto")
model.eval()

In [ ]:
ruta_docx = "/content/chatbottext.docx"
print("Procesando documento...")
parrafos = leer_docx_estructurado(ruta_docx)
chunks = extraer_chunks_por_titulo(parrafos)

# Guardar los chunks generados
guardar_chunks_en_txt(chunks)

print(f"Calculando embeddings")
embeddings_chunks = modelo_embeddings.encode(chunks, convert_to_tensor=True)


Ejemplo uso del chatbot

In [ ]:
pregunta = "What documents are needed to check in?"
respuesta = responder_con_busqueda_semantica(pregunta, chunks, embeddings_chunks)

print("Respuesta del chatbot:")
print(respuesta)


INTERFAZ GRAFICA

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def responder_interfaz(pregunta_usuario):
    respuesta = responder_con_busqueda_semantica(pregunta_usuario, chunks, embeddings_chunks)
    return respuesta

demo = gr.Interface(
    fn=responder_interfaz,
    inputs=gr.Textbox(lines=2, placeholder="Haz tu pregunta aqui..."),
    outputs="text",
    title="Chatbot",
    description="Este chatbot responde en base a los terminos y condiciones de la aerolínea"
)

demo.launch()
